# S&P 500 Returns Prediction — Comprehensive Analysis & Pipeline


**Reference:** Insights and explorations are derived from the EDA notebook [Hull Tactical: Complete EDA Deep Dive](https://www.kaggle.com/code/ahsuna123/hull-tactical-complete-eda-deep-dive).


---


## 1. Dataset Overview
- **Rows:** 8,990  
- **Features:** 98 (94 numeric, 4 special)  
- **Target:** `forward_returns`  
- **Feature Categories:**
  - **Market Dynamics:** M1–M13  
  - **Macro Economic:** E1–E20  
  - **Interest Rate:** I1–I9  
  - **Price Valuation:** P1–P13  
  - **Volatility:** V1–V13  
  - **Sentiment:** S1–S12  
  - **Dummy Binary:** D1–D9  
  - **Special:** 4  


✅ Rich in macroeconomic and market features; some categories show **high internal correlations**.


---


## 2. Target Variable Insights
- `forward_returns`:
  - Not normally distributed (Jarque-Bera p-value = 0)  
  - Stationary (ADF test p-value = 0)  
  - Weak autocorrelation: lag 1 ≈ -0.045, lag 5 ≈ -0.024  
  - Skewness ≈ -0.176, Kurtosis ≈ 2.19 → slightly platykurtic  
  - Maximum drawdown ≈ -0.492  


**Implications:**  
- Use robust methods (tree-based, quantile regression).  
- Temporal features can be leveraged due to stationarity.


---


## 3. Feature Correlation & Stability
- Most features weakly correlated with target (<0.07).  
- High multicollinearity within:
  - Macro_Economic, Interest_Rate, Price_Valuation, Volatility, Sentiment.  
- Feature stability over time:
  - **Stable:** Dummy_Binary, M1  
  - **Unstable:** Macro_Economic (E11, E12), Price_Valuation (P12), Volatility & Sentiment (V10–V12, S12)  


**Implications:**  
- Non-linear models preferred.  
- Unstable features should be transformed (lag/rolling means, smoothing).


---


## 4. Outlier Analysis
- Significant outliers in Macro_Economic and Price_Valuation features.  
- Recommendation: Winsorization, clipping, or robust scaling.


---


## 5. Feature Engineering Strategy
- **Lag Features:** M1–M10, D1–D5, V1–V5 → lag1, lag3  
- **Rolling Features:** Volatility features → rolling mean/std (window=5)  
- **Interaction Features:** Selected pairs between Macro_Economic and Market_Dynamics features  
- **Outlier Handling:** Clipping based on category-specific bounds  
- **Dimensionality Reduction:** PCA (n_components=50) to reduce high correlation noise  
- **Feature Selection:** SelectKBest (mutual information) when PCA is not used


---


## 6. Modeling Approach
- **Ensemble:** LGBM + XGBoost + Random Forest  
- **Hyperparameters:** Reduced for fast inference (n_estimators ≤ 300, max_depth limited)  
- **Weighting:** LGBM 0.4, XGB 0.35, RF 0.25 (renormalized if a model fails)  
- **Scaling:** RobustScaler applied to features  
- **Training/Inference Flow:**
  - Fit scalers and PCA during training  
  - Handle outliers for each batch  
  - Create lag/rolling/interaction features consistently for training & prediction  
  - Maintain recent feature history for single-row lag computations  


**Advantages:**
- Robust to outliers  
- Handles weakly correlated features and non-linear effects  
- Maintains consistent feature structure between training and inference


---


## 7. Pipeline Usage
- **Training:** `pipeline.fit_from_file(train_path)`  
- **Prediction:** `predict(pl.DataFrame)` for batch inference  
- **Saving/Loading:** `pipeline.save_model('sp500_model.pkl')` / `pipeline.load_model('sp500_model.pkl')`  
- **Server Deployment:** `kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)`  


**Note:** Polars is used for batch input; features are converted to Pandas internally.


---


## 8. Key Recommendations
1. Start modeling with **tree-based ensembles** (LGBM, XGBoost, RF)  
2. Use **PCA or feature selection** to reduce dimensionality and multicollinearity  
3. Include **lag and rolling features** for unstable categories  
4. Apply **robust scaling/clipping** to mitigate extreme outliers  
5. Implement **time-series aware validation** (avoid random splits)  
6. Monitor **feature stability** for future iterations and potential new engineered features


---


**Summary:**  
This pipeline integrates insights from EDA, feature stability analysis, and target characteristics into an end-to-end, deployable model for predicting S&P 500 forward returns.


In [1]:
import os
import pandas as pd
import polars as pl
import numpy as np
import gc
import pickle
import torch
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score
from sklearn.base import BaseEstimator, RegressorMixin
import lightgbm as lgb
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

import kaggle_evaluation.default_inference_server
from model_my_own import SP500Predictor

class SP500PredictorWrapper(BaseEstimator, RegressorMixin):
    """sklearn-compatible wrapper for SP500Predictor"""
    def __init__(self, feature_dim=None, d_model=64, num_heads=8, num_layers=4,
                 d_ff=256, dropout=0.1, output_dim=32, transformer_epochs=10,
                 transformer_lr=1e-3, combine_features=True, xgb_params=None,
                 # -----------------
                 # MODIFICATION: Added batch size
                 transformer_batch_size=256
                 # -----------------
                 ):
        self.feature_dim = feature_dim
        self.d_model = d_model
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.d_ff = d_ff
        self.dropout = dropout
        self.output_dim = output_dim
        self.transformer_epochs = transformer_epochs
        self.transformer_lr = transformer_lr
        self.combine_features = combine_features
        self.xgb_params = xgb_params
        # -----------------
        # MODIFICATION: Store batch size
        self.transformer_batch_size = transformer_batch_size
        # -----------------
        self.model = None
    def fit(self, X, y):
        """Fit the SP500Predictor model"""
        if self.feature_dim is None:
            self.feature_dim = X.shape[1]
        
        # Initialize the model
        self.model = SP500Predictor(
            feature_dim=self.feature_dim,
            d_model=self.d_model,
            num_heads=self.num_heads,
            num_layers=self.num_layers,
            d_ff=self.d_ff,
            dropout=self.dropout,
            output_dim=self.output_dim,
            xgb_params=self.xgb_params,
            combine_features=self.combine_features,
            # -----------------
            # MODIFICATION: Pass batch size to the model
            transformer_batch_size=self.transformer_batch_size
            # -----------------
        )
        
        # Convert to torch tensors - check for invalid values
        X_array = X.values if hasattr(X, 'values') else X
        y_array = y.values if hasattr(y, 'values') else y
        
        # Replace any remaining NaN/inf with 0
        X_array = np.nan_to_num(X_array, nan=0.0, posinf=0.0, neginf=0.0)
        y_array = np.nan_to_num(y_array, nan=0.0, posinf=0.0, neginf=0.0)
        
        X_tensor = torch.FloatTensor(X_array)
        
        # Train the model
        self.model.fit(
            X_tensor,
            y_array,
            transformer_epochs=self.transformer_epochs,
            transformer_lr=self.transformer_lr,
            verbose=False
        )
        
        return self
    def predict(self, X):
        """Make predictions"""
        if self.model is None:
            raise ValueError("Model has not been fitted yet")
        
        X_array = X.values if hasattr(X, 'values') else X
        X_array = np.nan_to_num(X_array, nan=0.0, posinf=0.0, neginf=0.0)
        X_tensor = torch.FloatTensor(X_array)
        
        return self.model.predict(X_tensor)
    def get_params(self, deep=True):
        """Get parameters for this estimator"""
        return {
            'feature_dim': self.feature_dim,
            'd_model': self.d_model,
            'num_heads': self.num_heads,
            'num_layers': self.num_layers,
            'd_ff': self.d_ff,
            'dropout': self.dropout,
            'output_dim': self.output_dim,
            'transformer_epochs': self.transformer_epochs,
            'transformer_lr': self.transformer_lr,
            'combine_features': self.combine_features,
            'xgb_params': self.xgb_params,
            # -----------------
            # MODIFICATION: Add to params
            'transformer_batch_size': self.transformer_batch_size
            # -----------------
        }
    def set_params(self, **params):
        """Set parameters for this estimator"""
        for key, value in params.items():
            setattr(self, key, value)
        return self

class RegularizedSP500Pipeline:
    """Simplified pipeline with regularization to prevent overfitting"""

    def __init__(self):
        self.scaler = RobustScaler()
       
        # Heavy regularization to prevent overfitting
        self.models = {
            'lgb': lgb.LGBMRegressor(
                objective='regression',
                metric='rmse',
                num_leaves=15,
                learning_rate=0.01,
                n_estimators=100,
                min_child_samples=20,
                subsample=0.7,
                colsample_bytree=0.7,
                reg_alpha=1.0,
                reg_lambda=1.0,
                random_state=42,
                verbose=-1  # Suppress output
            ),
            'xgb': xgb.XGBRegressor(
                objective='reg:squarederror',
                n_estimators=50,
                max_depth=5,
                learning_rate=0.03,
                min_child_weight=5,
                subsample=0.7,
                colsample_bytree=0.7,
                reg_alpha=1.0,
                reg_lambda=1.0,
                random_state=42,
                verbosity=0
            ),
            'sp500_transformer': SP500PredictorWrapper(
                d_model=64,
                num_heads=8,
                num_layers=4,
                d_ff=128,
                dropout=0.2,
                output_dim=16,
                transformer_epochs=5,
                transformer_lr=1e-3,
                combine_features=True,
                xgb_params={
                    'n_estimators': 50,
                    'max_depth': 5,
                    'learning_rate': 0.03,
                    'reg_alpha': 1.0,
                    'reg_lambda': 1.0
                }
            )
        }

        self.model_weights = {'lgb': 0.35, 'xgb': 0.35, 'sp500_transformer': 0.30}
        self.feature_names = None
        self.is_fitted = False

    def create_features(self, df):
        """Minimal feature engineering to reduce overfitting risk - FIXED VERSION"""
        df_features = df.copy()
        # Exclude target-related columns from features
        excluded_cols = ['date_id', 'forward_returns', 'is_scored', 'risk_free_rate', 'market_forward_excess_returns']
        # FIX: Get feature columns ONCE before creating any derived features
        original_feature_cols = [c for c in df.columns if c not in excluded_cols]
       
        print(f"  Original features: {len(original_feature_cols)}")
       
        # Only 1-2 simple lags - use 0 fill instead of forward fill
        for lag in [1, 2]:
            for col in original_feature_cols:  # FIX: Use original_feature_cols, not feature_cols
                df_features[f'{col}_lag{lag}'] = df_features[col].shift(lag).fillna(0)
       
        # Only one rolling window
        for col in original_feature_cols:  # FIX: Use original_feature_cols
            df_features[f'{col}_roll_mean_5'] = df_features[col].rolling(5, min_periods=1).mean()
       
        # Count total features after engineering
        final_feature_cols = [c for c in df_features.columns if c not in excluded_cols]
        print(f"  Features after engineering: {len(final_feature_cols)}")
       
        return df_features

    def prepare_features(self, df, is_training=False):
        print("Creating features...")
        df_enh = self.create_features(df)
       
        # Exclude target-related columns
        excluded_cols = ['date_id', 'forward_returns', 'is_scored', 'risk_free_rate', 'market_forward_excess_returns']
        feature_cols = [c for c in df_enh.columns if c not in excluded_cols]
        X = df_enh[feature_cols].copy()

        print(f"Handling missing values and outliers...")
        # First fill NaN before computing quantiles
        X = X.fillna(X.median()).fillna(0)
       
        # Then clip outliers
        for col in X.columns:
            try:
                q05, q95 = X[col].quantile(0.05), X[col].quantile(0.95)
                if not np.isnan(q05) and not np.isnan(q95):
                    X[col] = X[col].clip(q05, q95)
            except:
                pass  # Skip columns that can't be clipped

        # Final NaN check
        X = X.fillna(0)

        if is_training:
            print(f"Fitting scaler on {X.shape[0]} samples with {X.shape[1]} features...")
            X_scaled = self.scaler.fit_transform(X)
            self.feature_names = list(X.columns)
        else:
            # Ensure all expected features exist
            for col in self.feature_names:
                if col not in X.columns:
                    X[col] = 0
            X = X[self.feature_names]
            X_scaled = self.scaler.transform(X)

        return pd.DataFrame(X_scaled, columns=self.feature_names, index=df.index)

    def fit_from_file(self, train_path, target_col='forward_returns'):
        print("="*60)
        print("STARTING TRAINING PIPELINE")
        print("="*60)
        print(f"\n[1/6] Loading training data from {train_path}...")
        # Read CSV with proper NA handling
        df = pd.read_csv(train_path, na_values=['', 'NA', 'N/A', 'null']).iloc[1000:] # Skip first 1000 rows as per original logic
        print(f"  ✓ Loaded {df.shape[0]} rows × {df.shape[1]} columns")
        y = df[target_col].fillna(0)
        print(f"  ✓ Target variable: {target_col} (mean={y.mean():.6f}, std={y.std():.6f})")
       
        self.y_mean, self.y_std = 0, 1
       
        print(f"\n[2/6] Preparing features...")
        X_full = self.prepare_features(df, is_training=True)
        print(f"  ✓ Final feature matrix: {X_full.shape[0]} rows × {X_full.shape[1]} features")

        # TimeSeriesSplit CV
        tscv = TimeSeriesSplit(n_splits=5)
        cv_results = {name: {'rmse': [], 'r2': [], 'var_y': []} for name in self.models}

        print(f"\n[3/6] Running TimeSeriesSplit Cross-Validation (5 folds)...")
        for fold, (train_idx, val_idx) in enumerate(tscv.split(X_full), 1):
            X_train, X_val = X_full.iloc[train_idx], X_full.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            print(f"\n  Fold {fold}/5: train_size={len(X_train)}, val_size={len(X_val)}")

            for name, model in self.models.items():
                print(f"    [{name}] Training...", end=' ', flush=True)
                # Clone model to avoid state carryover
                fold_models = {} # Store clones for cleanup
                if name == 'lgb':
                    model_clone = lgb.LGBMRegressor(**model.get_params())
                elif name == 'xgb':
                    model_clone = xgb.XGBRegressor(**model.get_params())
                else:  # sp500_transformer
                    model_clone = SP500PredictorWrapper(**model.get_params())
                #print(f"fititng {name} model")
                model_clone.fit(X_train, y_train)
                preds = model_clone.predict(X_val)
                rmse = np.sqrt(np.mean((preds - y_val)**2))
                r2 = r2_score(y_val, preds)
                var_y = np.var(y_val)
                cv_results[name]['rmse'].append(rmse)
                cv_results[name]['r2'].append(r2)
                cv_results[name]['var_y'].append(var_y)
                print(f"RMSE={rmse:.6f}, R²={r2:.6f}")
                fold_models[name] = model_clone  # Store for cleanup
            
            print(f"    [Memory] Clearing memory for fold {fold}...")
            
            # Explicitly delete the cloned models and large tensors
            del fold_models, X_train, X_val, y_train, y_val, preds
            
            # Run Python's garbage collector
            gc.collect()
            
            # Empty the PyTorch CUDA cache (the most important step)
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                
                print(f"    [Memory] Fold {fold} memory cleared.")

        # Summary statistics
        print(f"\n[4/6] Cross-Validation Results Summary:")
        print("="*60)
        for name, stats in cv_results.items():
            print(f"\n{name.upper()}:")
            for metric in ['rmse','r2','var_y']:
                values = stats[metric]
                if values:
                    print(f"  {metric.upper()}: mean={np.mean(values):.6f}, std={np.std(values):.6f}")

        # Check for overfitting
        for name, stats in cv_results.items():
            if stats['r2']:
                avg_r2 = np.mean(stats['r2'])
                if avg_r2 > 0.5:
                    print(f"\n⚠️  WARNING: {name} shows high R² ({avg_r2:.3f}) - likely overfitting!")
                if avg_r2 < 0:
                    print(f"\n⚠️  WARNING: {name} has negative R² ({avg_r2:.3f}) - worse than baseline!")

        # Refit models on full dataset
        print(f"\n[5/6] Refitting models on full dataset...")
        for name, model in self.models.items():
            try:
                print(f"  [{name}] Training on full data...", end=' ', flush=True)
                model.fit(X_full, y)
                print("✓")
            except Exception as e:
                print(f"✗ Error: {e}")
                import traceback
                traceback.print_exc()
                self.model_weights[name] = 0

        # Adjust weights based on inverse RMSE
        print(f"\n[6/6] Calculating ensemble weights...")
        inv_rmse = {}
        for k in self.models:
            if cv_results[k]['rmse']:
                mean_rmse = np.mean(cv_results[k]['rmse'])
                inv_rmse[k] = 1.0 / mean_rmse if mean_rmse > 0 else 0
            else:
                inv_rmse[k] = 0
       
        total = sum(inv_rmse.values())
        if total > 0:
            self.model_weights = {k: v/total for k,v in inv_rmse.items()}

        print(f"  Final model weights:")
        for name, weight in self.model_weights.items():
            print(f"    {name}: {weight:.3f}")

        self.is_fitted = True
        print("\n" + "="*60)
        print("✓ TRAINING COMPLETED SUCCESSFULLY!")
        print("="*60)
        return self

    def predict_batch(self, df_batch):
        if not self.is_fitted:
            raise ValueError("Model not fitted")
        X_full = self.prepare_features(df_batch, is_training=False)
        preds = np.zeros(len(X_full))
        for name, model in self.models.items():
            weight = self.model_weights.get(name, 0)
            if weight > 0:
                preds += weight * model.predict(X_full)
        return preds[0] if len(preds)==1 else preds

    def save_model(self, filepath='regularized_sp500_model.pkl'):
        """Save the entire pipeline including PyTorch models"""
        models_to_save = {}
        for name, model in self.models.items():
            if name == 'sp500_transformer' and model.model is not None:
                models_to_save[name] = {
                    'params': model.get_params(),
                    'transformer_state': model.model.transformer.state_dict(),
                    'xgboost_model': model.model.xgboost_model,
                    'combine_features': model.model.combine_features
                }
            else:
                models_to_save[name] = model
       
        model_data = {
            'models': models_to_save,
            'model_weights': self.model_weights,
            'scaler': self.scaler,
            'feature_names': self.feature_names,
            'is_fitted': self.is_fitted,
            'y_mean': self.y_mean,
            'y_std': self.y_std
        }
        with open(filepath,'wb') as f:
            pickle.dump(model_data,f)
        print(f"Model saved to {filepath}")

    def load_model(self, filepath='regularized_sp500_model.pkl'):
        """Load the entire pipeline including PyTorch models"""
        with open(filepath,'rb') as f:
            model_data = pickle.load(f)
       
        # Restore models
        self.models = {}
        for name, model_obj in model_data['models'].items():
            if name == 'sp500_transformer' and isinstance(model_obj, dict):
                wrapper = SP500PredictorWrapper(**model_obj['params'])
                wrapper.feature_dim = model_obj['params']['feature_dim']
               
                wrapper.model = SP500Predictor(
                    feature_dim=wrapper.feature_dim,
                    d_model=wrapper.d_model,
                    num_heads=wrapper.num_heads,
                    num_layers=wrapper.num_layers,
                    d_ff=wrapper.d_ff,
                    dropout=wrapper.dropout,
                    output_dim=wrapper.output_dim,
                    xgb_params=wrapper.xgb_params,
                    combine_features=model_obj['combine_features']
                )
               
                wrapper.model.transformer.load_state_dict(model_obj['transformer_state'])
                wrapper.model.xgboost_model = model_obj['xgboost_model']
                wrapper.model.combine_features = model_obj['combine_features']
               
                self.models[name] = wrapper
            else:
                self.models[name] = model_obj
       
        self.model_weights = model_data['model_weights']
        self.scaler = model_data['scaler']
        self.feature_names = model_data['feature_names']
        self.is_fitted = model_data['is_fitted']
        self.y_mean = model_data.get('y_mean',0)
        self.y_std = model_data.get('y_std',1)
        print(f"Model loaded from {filepath}")


# DON'T auto-run training in notebook - let user control it
print("✓ Pipeline classes loaded successfully!")
print("\nTo train the model, run:")
print("  pipeline = RegularizedSP500Pipeline()")
print("  pipeline.fit_from_file('train.csv')")
print("\nTo use for inference:")
print("  def predict(test: pl.DataFrame) -> float:")
print("      test_df = test.to_pandas()")
print("      return float(pipeline.predict_batch(test_df))")

✓ Pipeline classes loaded successfully!

To train the model, run:
  pipeline = RegularizedSP500Pipeline()
  pipeline.fit_from_file('train.csv')

To use for inference:
  def predict(test: pl.DataFrame) -> float:
      test_df = test.to_pandas()
      return float(pipeline.predict_batch(test_df))


## Training the Pipeline


Now you can manually control the training process:


### Option 1: Train on Full Dataset
```python
pipeline = RegularizedSP500Pipeline()
pipeline.fit_from_file('train.csv')
pipeline.save_model('sp500_model.pkl')
```


### Option 2: Quick Test with Subset (Recommended First)
```python
# Test with first 1000 rows to verify everything works
import pandas as pd
df_small = pd.read_csv('train.csv', nrows=1000)
df_small.to_csv('train_small.csv', index=False)


pipeline = RegularizedSP500Pipeline()
pipeline.fit_from_file('train_small.csv')
```


### Option 3: For Kaggle Submission
After training, create the inference function:
```python
def create_prediction_function():
    pipeline = RegularizedSP500Pipeline()
    pipeline.load_model('sp500_model.pkl')  # Load pre-trained model
    
    def predict(test: pl.DataFrame) -> float:
        try:
            test_df = test.to_pandas()
            pred = pipeline.predict_batch(test_df)
            signal = np.clip(pred * 50.0 + 1.0, 0.0, 2.0)
            return float(signal)
        except Exception as e:
            print(f"Prediction error: {e}")
            return 1.0
    return predict
```


In [2]:
# Use the FAST optimized version
pipeline = RegularizedSP500Pipeline()
pipeline.fit_from_file('kaggle/train.csv')
#pipeline.save_model('features/sp500_model.pkl')

STARTING TRAINING PIPELINE

[1/6] Loading training data from kaggle/train.csv...
  ✓ Loaded 7990 rows × 98 columns
  ✓ Target variable: forward_returns (mean=0.000478, std=0.010836)

[2/6] Preparing features...
Creating features...
  Original features: 94
  Features after engineering: 376
Handling missing values and outliers...
Fitting scaler on 7990 samples with 376 features...
  ✓ Final feature matrix: 7990 rows × 376 features

[3/6] Running TimeSeriesSplit Cross-Validation (5 folds)...

  Fold 1/5: train_size=1335, val_size=1331
    [lgb] Training... RMSE=0.012821, R²=-0.005594
    [xgb] Training... RMSE=0.012821, R²=-0.005594
    [sp500_transformer] Training... RMSE=0.012821, R²=-0.005594
    [Memory] Clearing memory for fold 1...
    [Memory] Fold 1 memory cleared.

  Fold 2/5: train_size=2666, val_size=1331
    [lgb] Training... RMSE=0.012126, R²=0.001386
    [xgb] Training... RMSE=0.012132, R²=0.000270
    [sp500_transformer] Training... RMSE=0.012137, R²=-0.000414
    [Memory] 